# Euler and RK 4

Reusable class to calculate approximations using forward euler and rk4. SDC inherits.

In [2]:
import numpy as np

class SchemesPy: 
    soln = []
    
    def __init__(self, v_field, start, stop, h, init_conditions):
        self.vector_field = v_field
        self.ics = init_conditions
        self.a = start
        self.b = stop
        self.times = np.arange(self.a,self.b,h)  
        self.h = h
          
    def prep_preds(scheme):
        def wrapper(self):
            n = self.vector_field(self.ics, 
                                  self.times[0]).size
            x = np.zeros(((self.times).size,
                          n))
            x[0,:] = self.ics
            self.soln = scheme(self,x).flatten()
            return self.times, self.soln
        return wrapper
 
    @prep_preds
    def euler(self, x):
        for k, t in enumerate(self.times[:-1]):
            x[k+1,:] = x[k,:]+self.h*self.vector_field(x[k,:],t)
        return x
    
    @prep_preds
    def rk4(self, x):
        for k, t in enumerate(self.times[:-1]):
            f1 = self.vector_field(x[k,:],t)
            f2 = self.vector_field(x[k,:]+0.5*self.h*f1,t+0.5*self.h)
            f3 = self.vector_field(x[k,:]+0.5*self.h*f2,t+0.5*self.h)
            f4 = self.vector_field(x[k,:]+self.h*f3,t+self.h)
            x[k+1,:] = x[k,:]+self.h/6*(f1+2*f2+2*f3+f4)
        return x
    
    def __str__(self):
        return np.dstack((self.times, self.soln))
        

In [3]:
dy_dt = lambda y,t : 1 - t + 4*y
y_pred = SchemesPy(v_field=dy_dt, 
                   start=0, 
                   stop=2, 
                   h=0.01, 
                   init_conditions=1)

y_pred.rk4()[1]

array([1.00000000e+00, 1.05096279e+00, 1.10390339e+00, 1.15890251e+00,
       1.21604415e+00, 1.27541577e+00, 1.33710836e+00, 1.40121664e+00,
       1.46783921e+00, 1.53707867e+00, 1.60904181e+00, 1.68383981e+00,
       1.76158833e+00, 1.84240781e+00, 1.92642357e+00, 2.01376605e+00,
       2.10457101e+00, 2.19897977e+00, 2.29713940e+00, 2.39920297e+00,
       2.50532981e+00, 2.61568574e+00, 2.73044335e+00, 2.84978228e+00,
       2.97388950e+00, 3.10295960e+00, 3.23719513e+00, 3.37680689e+00,
       3.52201428e+00, 3.67304567e+00, 3.83013875e+00, 3.99354088e+00,
       4.16350956e+00, 4.34031276e+00, 4.52422942e+00, 4.71554982e+00,
       4.91457613e+00, 5.12162284e+00, 5.33701725e+00, 5.56110005e+00,
       5.79422581e+00, 6.03676359e+00, 6.28909749e+00, 6.55162732e+00,
       6.82476916e+00, 7.10895610e+00, 7.40463890e+00, 7.71228672e+00,
       8.03238786e+00, 8.36545055e+00, 8.71200376e+00, 9.07259804e+00,
       9.44780643e+00, 9.83822533e+00, 1.02444755e+01, 1.06672030e+01,
      

# Algorithm 4.1 (Spectral Deferred Correction).

###  Compute initial approximation
For non-stiff/stiff problems, use the forward/backward Euler method to compute an approximate solution φ_i ≈ φ(s_i ) at the nodes s1, ..., sm on
the interval [a,b] 
###  Compute successive corrections.
    do j = 1,...,J
        1) Compute the approximate residual function 
            σ(φ[j−1]).
        2a) For non-stiff problems, compute 
            δ[j] = C_exp(G,σ(φ[j−1]))
        2b) For stiff problems, compute 
            δ[j] = C_imp(G, σ(φ[j−1]))
        3) Update the approximate solution 
            φ[j] = φ[j−1] + δ[j].
    enddo

In [4]:
import scipy as sp

class SDC(SchemesPy):
    
    """ For non-stiff problems which are numerically stable. 
    Stiff problems are when certain numerical methods for solving 
        the equation are numerically unstable, unless the step size is 
        taken to be extremely small.
    """
    def sdc(self):
        psi = []
        # Initial approximation
        ts, f_euler = self.euler()
        psi.append(f_euler)
        
        m=10
        
        # Gauss Legenre nodes, s1 .. sm on [a,b]
        gl_nodes = [(self.b - self.a)/2 *x + (self.b+self.a)/2
                    for x in np.polynomial.legendre.leggauss(m)]
        
        c = lambda i,t : np.product([(t - ts[j])/(ts[i]-ts[j]) 
                                     for j in range(1,m+1) if i!=j])
        
        # Lagrange Interpolant
        L_m = lambda psi, t : np.sum([ c(i,t)*psi[i] 
                                      for i in range(1,m+1)])
        # Integration matrix
        S_m = lambda f : [sp.integrate(L_m(f,t), -1, t_i, args=(f))
                          for t_i in ts]
        
        # Successive corrections
        for j in range(1, len(f_euler)):            
            pass
            # Residual = S^m _F(psi[j-1])- psi(j-1) + ^psi_a
            # residual = S_m()
            
            # For non-stiff problems (C_exp)
            
            # Update approximate solution

In [5]:
y = SDC(v_field=dy_dt, 
        start=0, 
        stop=2, 
        h=0.01, 
        init_conditions=1).sdc()

## Convergence
N is number of steps to take
T is length of interval, [a,b]
Timestep h= T/N